# Modelos Generativos

Para a análise de sentimento com modelos generativos, embora tenham sido realizados vários testes, optou-se por apresentar apenas os melhores resultados, que também são destacados no nosso relatório. Inicialmente, foi conduzido um estudo exploratório no ficheiro `notebook-dev.csv`, com o objetivo de selecionar o modelo com melhor desempenho. Os modelos apresentados a seguir estão ordenados por desempenho decrescente:

1. Gervásio 7B PT-BR
2. Granite-3.2-8B-Instruct
3. Mistral-7B-Instruct-v0.1
4. GlórIA 1.3B

Mantiveram-se constantes os seguintes parâmetros: `batch_size = 8`, `número máximo de tokens = 3`, e o formato de saída em texto.  
Como último passo, aplicou-se o modelo com melhor desempenho para realizar o labeling das reviews no ficheiro `notebook-challenge.txt`.



In [ ]:
"""
É necessário usar a GPU, e instalar algumas dependências:
pip install unsloth accelerate transformers
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
"""

## Gervásio 7B PT-BR

> Devido à importância da adaptação linguística para a análise de sentimento, utilizou-se o modelo Gervásio, um decodificador otimizado para português. Existem duas variantes disponíveis: uma ajustada ao português europeu (PT-PT) e outra ao português do Brasil (PT-BR). Considerando que a maioria dos comentários analisados se mantêm em português do Brasil, optou-se pela utilização da versão PT-BR.

> A utilização de um prompt melhorado, com exemplos adaptados a partir das reviews analisadas, foi crucial para o desempenho do modelo, aumentando a precisão das respostas. A principal desvantagem observada foi o esforço computacional necessário para a execução desta abordagem.


In [ ]:
import os, time, torch, pandas as pd
from huggingface_hub import login
from unsloth import FastLanguageModel
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Verificar se temos GPU
print("GPU disponível:", torch.cuda.is_available())
print("Nome da GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Nenhuma")

# Já que este exercício trata-se de fazer apenas inferência, não precisamos destes parameters, que são usados para fine-tuning
os.environ["TORCHINDUCTOR_DISABLE"] = "1"
os.environ["TORCHDYNAMO_DISABLE"] = "1"

# Autenticar Hugging Face
login()

In [ ]:
# Leitura do datasets:
file_path_devel = "notebooks-train.csv"
de_devel = pd.read_csv(file_path_devel, sep='\t')

In [ ]:
# Carregar o modelo e tokenizer (Gervásio via Hugging Face)
def carregar_modelo():
    model_name = "PORTULAN/gervasio-7b-portuguese-ptbr-decoder"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True)
    return model, tokenizer

# Função para especificar o prompt, e a resposta do modelo
def classificar_sentimento_batch(textos, pipe, batch_size):
    resultados = []
    for i in range(0, len(textos), batch_size):
        batch = textos[i:i + batch_size]
        prompts = [
            f"""Classifica o sentimento da seguinte review de um utilizador sobre o desempenho de um computador.

            INSTRUÇÕES:
            - Responde apenas com UMA palavra: "positivo" ou "negativo".
            - Não escrevas mais nada. Não expliques.
            - Não uses pontuação, espaços ou outros símbolos.

            Exemplo 1:
            input: "Notebook bom e rápido, tem um HD espaçoso e uma configuração que atende muito bem para o trabalho diário. Recomendo a todos para estudos e tarefas normais, como acessar a internet e ver filmes em alta definição."
            Classificação: positivo

            Exemplo 2:
            input: "Atende as minhas necessidades. Muito bom. Roda bem os programas, tem um belo design.."
            Classificação: positivo

            Exemplo 3:
            input: "Eu fiz a compra mês passado e cancelei as vésperas de receber o produto. Motivo: fiz a compra em boleto, no valor de R$ 2880,00, mas a empresa baixou o valor para R$ 2610,00 do produto um dia antes de chegar em minha casa. Publicaram até nas minhas redes sociais. É um direito da empresa, caso queira, dar de graça o produto, mas eu vejo como falta de respeito, já que não me deram a possibilidade de obter o desconto da promoção. O SAC é horrível e não há um setor ou opção que pudesse dar uma tratativa e salvar a venda, atribuindo o desconto aos clientes que desejassem dentro do prazo vigente da promoção. Pra ser bem claro, com um desconto promocional de R$ 270,00 reais em relação ao que eu paguei, a empresa diz em outras palavras 'olha aí otário, você perdeu dinheiro na compra'. Sendo assim, prefiro comprar numa empresa concorrente do que realizar uma nova compra com a <empresa>. Perdeu mais cliente mas vejo que a empresa nunca se importou em respeitar seus clientes."
            Classificação: negativo

            Exemplo 4:
            input: "Comprei o produto, após ver alguns vídeos na internet sobre seu funcionamento que parecia ser bem simples, porém não funciona com IOS11 e deveria já ter saído de circulação. Um crime contra o consumidor."
            Classificação: negativo

            input: "{texto}"
            Classificação:"""
            for texto in batch
        ]
        saidas = pipe(prompts, max_new_tokens=3, do_sample=False, pad_token_id=pipe.tokenizer.eos_token_id)
        for texto, saida in zip(batch, saidas):
            resposta = saida[0]['generated_text'].strip().lower().strip(".!?,;: \n")
            print(f"\nFrase: {texto}")
            print(f"Classificação: {resposta}")
            if "pos" in resposta:
                resultados.append("pos")
            elif "neg" in resposta:
                resultados.append("neg")
            else:
                resultados.append(resposta)
    return resultados

# Aplicar a classificação ao DataFrame
def aplicar_classificacao(df):
    model, tokenizer = carregar_modelo()
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=False)
    df['sentiment_pred'] = classificar_sentimento_batch(df['review_text'].tolist(), pipe, batch_size=8)
    return df

In [ ]:
# Aplicar classificação, e registar o tempo de execução
start_time = time.time()
de_devel = aplicar_classificacao(de_devel)
end_time = time.time()

print("Classificação concluída em", round(end_time - start_time, 2), "segundos")

In [ ]:
# Contar previsões corretas (só conta se for "pos" ou "neg") para cálculo da accuracy
corretos = ((de_devel['sentiment_pred'] == de_devel['sentiment']) & de_devel['sentiment_pred'].isin(['pos', 'neg'])).sum()
total = len(de_devel)
accuracy = corretos / total if total > 0 else 0
print("Accuracy do modelo: ", round(accuracy * 100, 2), "%")

In [ ]:
# Código opcional: guardar num pickle file
de_devel.to_pickle("df_gervasio_delve.pkl")

### Estudo das Classificações Erradas

> Foram identificadas e estudadas duas situações principais além das classificações corretas: respostas classificadas incorretamente e respostas incoerentes.

In [ ]:
pd.set_option('display.max_colwidth', None)

# Leitura do ficheiro com as reviews rotuladas, após classificação
df_gervasio = pd.read_pickle("df_gervasio_delve.pkl")

In [ ]:
df_gervasio['sentiment_pred'].value_counts()

,count
sentiment_pred,
pos,1190
neg,305
não especific,2
"não, não",1
não relacionado,1
não receb,1


#### Classifições Incorretas

In [ ]:
# Ver os exemplos mal classificados, que pelo menos tiveram labels bem formatadas em "pos" ou "neg"
df_unmatched = df_gervasio[(df_gervasio['sentiment'] != df_gervasio['sentiment_pred']) & (df_gervasio['sentiment_pred'].str.contains('pos', 'neg'))]
print(f"Número de linhas: {df_unmatched.shape[0]}")
df_unmatched[['review_text','sentiment', 'sentiment_pred']].head()

Número de linhas: 19


,review_text,sentiment,sentiment_pred
49,Prós: Marca renomada/possível boa durabilidade. Contras: processador lento/ aparelho pesado/ bateria dura pouco/ não vem com Office,neg,pos
133,"A entrega da <empresa> foi bem rápida, o produto é bonito mas só funciona se estiver na tomada, não carrega a bateria. Pedi troca e a loja prontamente recolheu o produto com defeito e vai enviar outro.",neg,pos
183,"Óbvio que é uma máquina limitada, mas por enquanto tem sido útil para meu principal uso que é o produzir trabalhos acadêmidos (usando Word, Excel e ppt) e pesquisa de informações na internet. As vezes tem uma certa lentidão... fica meio travado.",neg,pos
408,Comprei o not quando fui jogar observei que a taxa de fps tava em 30 sendo que ele termina boa placa de vídeo capas de chegar até 192 de ms realizei tudo que meus amigos falaram att entre outros mais permanece assim Américas oque eu posso fazer,neg,pos
564,"Optei pela marca mas tive a pior decepção de minha vida! O primeiro que chegou travou na configuração inicial, precisou ser trocado; com o segundo eu não tive a mesma sorte, pois gostaria que também fosse substituído. Resumindo, estou há um ano com essa bomba, não tenho mais garantia (coisa que não altera em nada, já que durante o período meus problemas nunca foram resolvidos) e logo vou ter que adquirir outro note, mas dessa vez, voltarei para o Acer, pois o meu de 8 anos ainda funciona razoavelmente bem.",neg,pos


> Após a análise dos exemplos classificados incorretamente, verificou-se que muitos dos textos apresentavam opiniões mistas, combinando comentários positivos e negativos, o que já tinha sido identificado no relatório como um fator de dificuldade para a correta classificação do sentimento.

> Exemplos como "Marca renomada/possível boa durabilidade" versus "processador lento/aparelho pesado" ilustram como a coexistência de prós e contras pode confundir o modelo, embora tenham sido separados como Prós e Contras pelo comentador. Outros casos apresentaram descrições técnicas, relatos de trocas de produtos ou avaliações moderadas com críticas e elogios simultâneos, tornando a definição do sentimento dominante particularmente ambígua.

> **Nota**: O modelo teve uma maior tendência a classificar erradamente *reviews* com *labels* reais negativas. Uma possível melhoria seria a de incluir mais exemplos negativos no *prompt*, desde que façam sentido para o modelo.


#### Respostas incoerentes

In [ ]:
# Ver os exemplos mal classificados, onde o output foi diferente de "pos" ou "neg"
df_unmatched_inc = df_gervasio[(df_gervasio['sentiment'] != df_gervasio['sentiment_pred']) & ~df_gervasio['sentiment_pred'].isin(['pos', 'neg'])]
df_unmatched_inc[['review_text','sentiment', 'sentiment_pred']].head()

,review_text,sentiment,sentiment_pred
143,Alguém sabe se é bom para rodar o Premiere?? Estou procurando um notebook para trabalho.,pos,não especific
178,Esse notebook é bom para jogar jogo de tiro fps em primeira pessoa?,pos,"não, não"
409,De acordo com as especificações técnicas do produto.,pos,não especific
488,"Vir com distro do linux faz o preço cair, e é só formatar e por windows.",pos,não relacionado
1316,"Recebi um e-mail pedindo para avaliar o produto, mas EU NÃO RECEBI E NÃO SEI ONDE ESTÁ O PRODUTO.",neg,não receb


> Já nas respostas incoerentes, o modelo não conseguiu atribuir uma etiqueta de sen-timento, sobretudo em textos que não expressavam uma opinião concreta sobre o produto, mas sim dúvidas, informações técnicas ou relatos sobre o processo de compra.

## Modelo Granite-3.2-8B-Instruct (Unsloth)



> O modelo Granite-3.2-8B-Instruct, disponibilizado via Unsloth, é uma versão otimizada do modelo Granite original da IBM, concebido para tarefas de geração e inferência. A principal vantagem da versão Unsloth é o ganho de performance, permitindo o dobro da velocidade de treino e inferência e uma redução significativa do consumo de memória VRAM, o que foi prático para este Trabalho.

> Mais informação sob o Unlosth e outros modelos otimizados por eles aqui: https://huggingface.co/unsloth


In [ ]:
# Inicializar novamente o dataset:
file_path_devel = "notebooks-train.csv"
de_devel = pd.read_csv(file_path_devel, sep='\t')

In [ ]:
# Carregar modelo Granite com Unsloth
def carregar_modelo():
    model_name = "unsloth/granite-3.2-8b-instruct-bnb-4bit"
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_name,
        max_seq_length = 2048,
        load_in_4bit = True,
        dtype = None
    )
    FastLanguageModel.for_inference(model)
    return model, tokenizer

def classificar_sentimento_batch(textos, pipe, batch_size):
    resultados = []
    for i in range(0, len(textos), batch_size):
        batch = textos[i:i+batch_size]
        prompts = [
            f"""
            Your task is to analyze the sentiment of the following sentence in Portuguese, which is a review about a computer.
            Respond with only one word: "positive" or "negative".
            Do not explain. Do not write anything else. This is the sentence: {texto}
            Answer:"""
            for texto in batch
        ]
        saidas = pipe(prompts, max_new_tokens = 3, do_sample = False, pad_token_id = pipe.tokenizer.eos_token_id)
        for texto, saida in zip(batch, saidas):
            resposta = saida[0]['generated_text'].strip().lower().strip(".!?,;: \n")
            print(f"\nFrase: {texto}")
            print(f"Classificação: {resposta}")
            if "pos" in resposta:
                resultados.append("pos")
            elif "neg" in resposta:
                resultados.append("neg")
            else:
                resultados.append(resposta)
    return resultados

# Aplicar a classificação ao DataFrame
def aplicar_classificacao(df):
    model, tokenizer = carregar_modelo()
    pipe = pipeline("text-generation", model = model, tokenizer = tokenizer, return_full_text = False)
    df['sentiment_pred'] = classificar_sentimento_batch(df['review_text'].tolist(), pipe, batch_size = 8)
    return df

In [ ]:
# Aplicar a classificação
start_time = time.time()
de_devel = aplicar_classificacao(de_devel)
end_time = time.time()
print("Classificação concluída em", round(end_time - start_time, 2), "segundos")

In [ ]:
# Contar previsões corretas (só conta se for "pos" ou "neg") para cálculo da accuracy
corretos = ((de_devel['sentiment_pred'] == de_devel['sentiment']) & de_devel['sentiment_pred'].isin(['pos', 'neg'])).sum()
total = len(de_devel)
accuracy = corretos / total if total > 0 else 0
print("Accuracy do modelo: ", round(accuracy * 100, 2), "%")

In [ ]:
# Código opcional: guardar num pickle file
de_devel.to_pickle("df_granite_delve.pkl")

> Embora o modelo não tenha sido o que apresentou o melhor desempenho, foram identificados, nos testes realizados, os mesmos tipos de classificações incorretas observados anteriormente. Por esse motivo, optou-se por não apresentar em detalhe os resultados deste modelo.

## Modelo Mistral-7B-Instruct-v0.1

> O modelo Mistral-7B-Instruct-v0.1 é um modelo desenvolvido pela Mistral AI, otimizado para tarefas de geração de texto e instrução. Embora também exista uma versão disponibilizada pela plataforma Unsloth, nesta análise utilizou-se a versão original por motivos de comparação.



In [ ]:
# Inicializar novamente o dataset:
file_path_devel = "notebooks-train.csv"
de_devel = pd.read_csv(file_path_devel, sep='\t')

In [ ]:
def carregar_modelo():
    model_name = "mistralai/Mistral-7B-Instruct-v0.1"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True)
    return model, tokenizer

In [ ]:
# Aplicar a classificação
start_time = time.time()
de_devel = aplicar_classificacao(de_devel)
end_time = time.time()
print("Classificação concluída em", round(end_time - start_time, 2), "segundos")

In [ ]:
# Contar previsões corretas (só conta se for "pos" ou "neg") para cálculo da accuracy
corretos = ((de_devel['sentiment_pred'] == de_devel['sentiment']) & de_devel['sentiment_pred'].isin(['pos', 'neg'])).sum()
total = len(de_devel)
accuracy = corretos / total if total > 0 else 0
print("Accuracy do modelo: ", round(accuracy * 100, 2), "%")

In [ ]:
# Código opcional: guardar num pickle file
de_devel.to_pickle("df_mistral_delve.pkl")

> Tal como nos outros modelos analisados, os mesmos tipos de erros — classificações incorretas e respostas incoerentes — também foram identificados no Mistral-7B-Instruct-v0.1. Juntamente com o Gervásio 7B PT-BR, o Mistral foi um dos modelos mais exigentes em termos de custo computacional, apresentando tempos de execução elevados para inferência, o que limitou a realização de mais testes exploratórios.


## Modelo GlórIA 1.3B

> O modelo GlórIA 1.3B é um modelo de linguagem desenvolvido para o português europeu, focado em tarefas de geração e inferência. Tendo um tamanho mais reduzido em comparação com os outros modelos testados, decidimos fazer a experiência e testar o modelo.


In [ ]:
# Inicializar novamente o dataset:
file_path_devel = "notebooks-train.csv"
de_devel = pd.read_csv(file_path_devel, sep='\t')

In [ ]:
# Carregar o modelo e tokenizer (Mistral via Hugging Face)
def carregar_modelo():
    model_name = "NOVA-vision-language/GlorIA-1.3B"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True)
    return model, tokenizer

In [ ]:
# Aplicar a classificação
start_time = time.time()
de_devel = aplicar_classificacao(de_devel)
end_time = time.time()
print("Classificação concluída em", round(end_time - start_time, 2), "segundos")

In [ ]:
# Contar previsões corretas (só conta se for "pos" ou "neg") para cálculo da accuracy
corretos = ((de_devel['sentiment_pred'] == de_devel['sentiment']) & de_devel['sentiment_pred'].isin(['pos', 'neg'])).sum()
total = len(de_devel)
accuracy = corretos / total if total > 0 else 0
print("Accuracy do modelo: ", round(accuracy * 100, 2), "%")

In [ ]:
# Código opcional: guardar num pickle file
de_devel.to_pickle("df_gloria_delve.pkl")

> Os resultados obtidos com o GlórIA 1.3B não corresponderam totalmente às expectativas, apresentando uma *accuracy* de aproximadamente 56%. Apesar deste desempenho, o modelo foi avaliado como promissor no benchmark CALAME-PT, que mede a capacidade de prever a continuação de textos em português com base num contexto dado. Acreditamos que, com mais experiências e ajustes de prompts, o desempenho poderá ser significativamente melhorado.
>
> **Nota:** Documentação oficial disponível em: https://huggingface.co/NOVA-vision-language/GlorIA-1.3B



## *Labeling* do *notebook-challenge.txt*

> Para esta tarefa, foi usado o modelo Gervásio, por ter alcançado os melhores resultados. O código adaptado encontra-se a seguir:

In [ ]:
# Leitura do notebook sem labels
file_path_challenge = "notebooks-challenge.txt"
with open(file_path_challenge, 'r', encoding='utf-8') as f:
    next(f)
    test_data = [line.strip() for line in f]
df_challenge = pd.DataFrame(test_data, columns=['review_text'])

In [ ]:
# Aplicar a classificação ao DataFrame
def aplicar_classificacao(df):
    model, tokenizer = carregar_modelo()
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=False)
    df['sentiment'] = classificar_sentimento_batch(df['review_text'].tolist(), pipe, batch_size=8)
    return df

In [ ]:
# Aplicar classificação
start_time = time.time()
df_challenge = aplicar_classificacao(df_challenge)
end_time = time.time()
print("Classificação concluída em", round(end_time - start_time, 2), "segundos")

In [ ]:
# Opcional: guardar num pickle e CSV file
df_challenge.to_pickle("df_challenge_gervasio.pkl")
df_challenge.to_csv("df_challenge_gervasio.csv", index=False)

> Embora o documento final foi entregue junto com o relatório, podemos observar que a rotulagem foi bem sucedida em termos do formato de saída do *prompt*, como observado nos resultados para este modelo.

In [ ]:
df_challenge_complete = pd.read_csv("notebook-challenge.csv", sep='\t')
df_challenge_complete.head()

,sentiment,review_text
0,pos,"O note é bom, mas não superou as expectativas ..."
1,pos,Ele corresponde ao valor pago. Custando em méd...
2,pos,"O notebook é muito bom, o Windows 10 ferra um ..."
3,pos,"O notebook tem uma tela enorme, muito útil par..."
4,pos,Muito bom ! Excelente notebook! Entrega rapida...


In [ ]:
df_challenge_complete['sentiment'].value_counts()

,count
sentiment,
pos,323
neg,92
não especific,2
não entreg,1
simplesmente,1
não receb,1
não relacionado,1


> Os dois tipos de classificações erradas observados no *notebook-delve.csv* eram de certa forma esperados, já que as reviews são semelhantes às *reviews* do *notebook-delve.csv*, com a maioria escritas em português brasileiro.
